<img src='https://radiant-assets.s3-us-west-2.amazonaws.com/PrimaryRadiantMLHubLogo.png' alt='Radiant MLHub Logo' width='300'/>

# Exploratory Data Analysis

This notebook walks you through the steps to exploratory data analysis.

In [2]:
from radiant_mlhub import Collection
import tarfile
import os
from pathlib import Path
import json

import datetime
import rasterio
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedShuffleSplit
import pickle

### Load data

In [3]:
df_train = pd.DataFrame()
for value in ['df_median_1','df_median_2']:
    print('Reading Train df:', value)
    #read the pickle file
    picklefile = open(value, 'rb')
    #unpickle the dataframe
    df_tmp = pickle.load(picklefile)
    #close file
    picklefile.close()
    df_train = pd.concat([df_train,df_tmp])
    print(len(df_tmp))

Reading Train df: df_median_1
3603
Reading Train df: df_median_2
3571


In [4]:
df_train

,field_id,20170401_B01,20170401_B02,20170401_B03,20170401_B04,20170401_B05,20170401_B06,20170401_B07,20170401_B08,20170401_B09,...,20171127_B05,20171127_B06,20171127_B07,20171127_B08,20171127_B09,20171127_B11,20171127_B12,20171127_B8A,20171127_CLM,label
0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,64.0,70.0,76.0,85.0,81.0,113.0,88.0,86.0,0.0,8
1,91.0,22.0,25.0,34.0,45.0,52.0,56.0,60.0,64.0,70.0,...,48.0,52.0,57.0,62.0,67.0,108.0,91.0,64.5,0.0,3
2,243.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38.0,63.0,72.0,76.0,83.0,86.0,64.0,79.0,0.0,4
3,286.0,13.0,16.0,23.0,30.0,37.0,44.0,49.0,53.0,57.0,...,38.0,54.0,60.0,65.0,67.0,72.0,49.0,66.0,0.0,2
4,308.0,21.0,29.0,47.0,65.0,74.0,78.0,83.0,85.0,88.0,...,78.0,81.0,86.0,92.0,90.0,122.0,93.0,94.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3566,122508.0,18.0,25.0,41.0,64.0,77.0,82.0,90.0,98.0,103.0,...,72.0,75.0,81.0,87.0,89.0,118.0,89.0,88.0,0.0,6
3567,122532.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,73.0,75.0,78.0,82.0,84.0,118.0,93.0,83.0,0.0,7
3568,122534.0,10.5,14.0,24.0,34.0,43.0,51.0,56.0,61.0,61.0,...,53.0,73.0,80.0,88.0,84.0,96.0,65.0,86.0,0.0,4
3569,122585.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,58.5,80.0,89.0,95.0,92.0,103.5,83.0,94.0,0.0,4


In [8]:
df_train['label'].value_counts()

4    2133
7    1220
2    1037
1     767
6     684
5     503
3     474
8     209
9     147
Name: label, dtype: int64

### Conclusions

- En algunas fechas hay probabilidad de Nubes
- Cuando hay probabilidad de nubes (= 255), se remplaza por valor NaN en las bandas en la fecha correspondiente
- Para calcular los valores NaN utilizamos interpolación de la serie temporal

## Building the Model

In [ ]:
# Each field has several pixels in the data. Here our goal is to build a Random Forest (RF) model using the average values
# of the pixels within each field. So, we use `groupby` to take the mean for each field_id
data_grouped = data.groupby('field_id').mean().reset_index()
data_grouped

In [ ]:
# Split train and test
# We use field_ids to split the data to train and test. Note that the test portion for training is different than the test 
# portion provided as part of the competition. 
train_per = 0.7

n_fields = len(data_grouped['field_id'])
np.random.seed(10)
train_fields = np.random.choice(data_grouped['field_id'], int(n_fields * train_per), replace=False)
test_fields = data_grouped['field_id'][~np.in1d(data_grouped['field_id'], train_fields)]

In [ ]:
X_train, X_test = data_grouped[data_grouped['field_id'].isin(train_fields)], data_grouped[data_grouped['field_id'].isin(test_fields)]
X_train = X_train.drop(columns=['label', 'field_id'])
X_test = X_test.drop(columns=['label', 'field_id'])
y_train, y_test = data_grouped[data_grouped['field_id'].isin(train_fields)]['label'], data_grouped[data_grouped['field_id'].isin(test_fields)]['label']

In [ ]:
# We ran a simple hyperparameter tuning for the number of trees, and concluded to use:
n_trees = 50

In [ ]:
# Fitting the RF model
rf = RandomForestClassifier(n_estimators = n_trees, random_state = 0, n_jobs = 3)
rf.fit(X_train, y_train.astype(int))

## Competition Test Data

In this part we will load the competition test data (which does not have labels) and predict the crop class for each field

In [ ]:
tile_ids_test = competition_test_df['tile_id'].unique()

In [ ]:
X_competition_test = np.empty((0, 2 * (n_obs-1)))
field_ids_test = np.empty((0, 1))

for tile_id in tile_ids_test:
    tile_df = competition_test_df[competition_test_df['tile_id']==tile_id]
    
    field_id_src = rasterio.open(tile_df[tile_df['asset']=='field_ids']['file_path'].values[0])
    field_id_array = field_id_src.read(1)
    field_ids_test = np.append(field_ids_test, field_id_array.flatten())
    
    tile_date_times = tile_df[tile_df['satellite_platform']=='s1']['datetime'].unique()
    
    X_tile = np.empty((256 * 256, 0))
    for date_time in tile_date_times[ : 4 * n_obs : n_obs]:
        vv_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VV')]['file_path'].values[0])
        vv_array = np.expand_dims(vv_src.read(1).flatten(), axis=1)
        
        vh_src = rasterio.open(tile_df[(tile_df['datetime']==date_time) & (tile_df['asset']=='VH')]['file_path'].values[0])
        vh_array = np.expand_dims(vh_src.read(1).flatten(), axis=1)
        
        X_tile = np.append(X_tile, vv_array, axis = 1)
        X_tile = np.append(X_tile, vh_array, axis = 1)
        
    X_competition_test = np.append(X_competition_test, X_tile, axis=0)

In [ ]:
data_test = pd.DataFrame(X_competition_test)
data_test['field_id'] = field_ids_test
data_test = data_test[data_test.field_id != 0]
data_test

In [ ]:
data_test_grouped = data_test.groupby('field_id').mean().reset_index()
data_test_grouped

In [ ]:
y_competition_prob = rf.predict_proba(data_test_grouped.drop(columns=['field_id']))

In [ ]:
# In this part we format the DataFrame to have column names and order similar to the sample submission file. 
pred_df = pd.DataFrame(y_competition_prob)
pred_df = pred_df.rename(columns={
    0:'Crop_ID_1',
    1:'Crop_ID_2', 
    2:'Crop_ID_3',
    3:'Crop_ID_4',
    4:'Crop_ID_5',
    5:'Crop_ID_6',
    6:'Crop_ID_7',
    7:'Crop_ID_8',
    8:'Crop_ID_9'
})
pred_df['field_id']=data_test_grouped['field_id']
pred_df = pred_df[['field_id', 'Crop_ID_1', 'Crop_ID_2', 'Crop_ID_3', 'Crop_ID_4', 'Crop_ID_5', 'Crop_ID_6', 'Crop_ID_7', 'Crop_ID_8', 'Crop_ID_9']]
pred_df

In [ ]:
# Write the predicted probabilites to a csv for submission
pred_df.to_csv('baseline_submission.csv', index=False)